# Imports
`lonboard` is designed to work effectively within Jupyter Notebooks. Please ensure you use Jupyter Notebook to follow along.  

Before starting, make sure you have the following packages installed:
- [`lonboard`](https://developmentseed.org/lonboard/) and its dependencies: `anywidget`, `geopandas`, `matplotlib`, `palettable`, `pandas` (which requires `numpy`), `pyarrow` and `shapely`.
- `requests`: For fetching data.
- `seaborn`: For statistical data visualization.

Additionally, the following modules are part of the Python standard library and do not require separate installation:

- `zipfile`
- `io`
- `sys`

In [1]:
import requests
import zipfile
import io
import sys

from lonboard import Map, SolidPolygonLayer
from lonboard.layer_extension import DataFilterExtension
import ipywidgets as widgets

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.ticker as mticker

from shapely.geometry import Polygon

import seaborn as sns

# Fetch Data
The population data is stored in a .csv file within a zip folder. We will fetch the data and extract it into a panadas dataframe.

In [2]:
url = "https://www.zensus2022.de/static/Zensus_Veroeffentlichung/Zensus2022_Bevoelkerungszahl.zip"
target_file = "Zensus2022_Bevoelkerungszahl_100m-Gitter.csv"

response = requests.get(url)

if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref: # Open the ZIP file in memory
        if target_file in zip_ref.namelist():
            with zip_ref.open(target_file) as csv_file:
                df_pop = pd.read_csv(csv_file, delimiter=';') 
        else:
            print(f"{target_file} not found in the ZIP archive.")
else:
    print(f"Failed to download file: {response.status_code}")

We can take a quick look at this data:

In [3]:
df_pop.head()

,GITTER_ID_100m,x_mp_100m,y_mp_100m,Einwohner
0,CRS3035RES100mN2689100E4337000,4337050,2689150,4
1,CRS3035RES100mN2689100E4341100,4341150,2689150,11
2,CRS3035RES100mN2690800E4341200,4341250,2690850,4
3,CRS3035RES100mN2691200E4341200,4341250,2691250,12
4,CRS3035RES100mN2691300E4341200,4341250,2691350,3


The data contains four as described in the [data decsription](https://www.zensus2022.de/static/Zensus_Veroeffentlichung/Datensatzbeschreibung_Bevoelkerungszahl_Gitterzellen.xlsx): 
- `GITTER_ID_100m`: cell identifier
- `x_mp_100m`: longitude of the cell centre 
- `y_mp_100m`: latitude of the cell centre
- `Einwohner`: population 

# Prepare Data

In [4]:
df_pop.rename(columns={'Einwohner': 'Population'}, inplace=True)
#df_pop['Population'] = pd.to_numeric(df_pop['Population'], errors='coerce') # coerce: any value that cannot be converted to a numeric type will be replaced with NaN

To visualize the data, we need to transform the coordinates into polygons. The dataset uses the [ETRS89-LAEA Europe (EPSG: 3035)](https://epsg.io/3035) coordinate reference system, which measures distances in meters and is suited for Europe. Each cell is 100m x 100m, and the coordinates represent the cell center.

In [5]:
def create_polygon(x, y, half_length=50):
    return Polygon([
        (x - half_length, y - half_length),
        (x + half_length, y - half_length),
        (x + half_length, y + half_length),
        (x - half_length, y + half_length)
    ])
    
gdf_population = gpd.GeoDataFrame(df_pop['Population'], geometry=df_pop.apply(lambda row: create_polygon(row['x_mp_100m'], row['y_mp_100m']), axis=1), crs = 'EPSG:3035')

In [6]:
# Convert the coordinate system to WGS84 (EPSG:4326)
gdf_population = gdf_population.to_crs(epsg=4326)

In [8]:
gdf_population.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3088037 entries, 0 to 3088036
Data columns (total 2 columns):
 #   Column      Dtype   
---  ------      -----   
 0   Population  int64   
 1   geometry    geometry
dtypes: geometry(1), int64(1)
memory usage: 47.1 MB


In [12]:
gdf_population['Population'].min()

np.int64(3)

In [ ]:
gdf_population['Population'] = gdf_population

In [7]:
gdf_population

Size of GeoDataFrame: 47.12 MB


,Population,geometry
0,4,"POLYGON ((10.21146 47.31529, 10.21278 47.31529..."
1,11,"POLYGON ((10.26565 47.31517, 10.26697 47.31517..."
2,4,"POLYGON ((10.26705 47.33047, 10.26837 47.33046..."
3,12,"POLYGON ((10.26707 47.33407, 10.26839 47.33407..."
4,3,"POLYGON ((10.26707 47.33497, 10.26839 47.33497..."
...,...,...
3088032,14,"POLYGON ((8.42288 55.02299, 8.42445 55.02301, ..."
3088033,4,"POLYGON ((8.41816 55.02383, 8.41972 55.02385, ..."
3088034,10,"POLYGON ((8.41972 55.02385, 8.42129 55.02387, ..."
3088035,3,"POLYGON ((8.42129 55.02387, 8.42285 55.02389, ..."


# Visualize Data using lonboard

To visualize the population data, we use the [`SolidPolygonLayer`](https://developmentseed.org/lonboard/latest/api/layers/solid-polygon-layer/). We choose this layer over `PolygonLayer` because it does not render polygon outlines and is more performant. 
We create the `SolidPolygonLayer` from the GeoDataFrame and add it to a `Map`:

In [ ]:
polygon_layer = SolidPolygonLayer.from_geopandas(
    gdf_population,
)

m = Map(polygon_layer) 

In [ ]:
m

## Set a colormap

We can look at the documentation for [`SolidPolygonLayer`](https://developmentseed.org/lonboard/latest/api/layers/solid-polygon-layer/) to see what other rendering options it allows. Let's set the fill color to something other than black:

In [ ]:
polygon_layer.get_fill_color = [11, 127, 171]

Blue is pretty, but let's make our map more insightful by coloring each area based on its population by a relevant characteristic.  Using a [colomap](https://matplotlib.org/stable/users/explain/colors/colormaps.html), we'll scale the population values between 0 and 1, making it easy to see and compare different population densities at a glance.

In [ ]:
colormap = mpl.colormaps["YlOrRd"]
normalizer = mpl.colors.Normalize(0, vmax=gdf_population['Population'].max()) #Normalize population to the 0-1 range
colors = colormap(normalizer(gdf_population['Population']), bytes=True)

In [ ]:
polygon_layer.get_fill_color = colors

It looks like the red colors aren’t showing up much on the map, which might suggest that our data is skewed. Let's check:

In [ ]:
def thousands_formatter(x, pos):
    return f'{int(x):,}'
    
sns.histplot(gdf_population['Population'], bins=100)

plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(thousands_formatter))
plt.show()

print("Skewness value:",gdf_population['Population'].skew())

The histogram and skewness show a strong positive skew—most areas have low populations, with just a few spots having really high numbers. This means our linear colormap isn’t showing much color variation. To make the differences stand out, let’s switch to a logarithmic colormap to better highlight population differences.

In [ ]:
normalizer = mpl.colors.LogNorm(vmin=gdf_population['Population'].min(), vmax=gdf_population['Population'].max()) # Normalize population to the 0-1 range on a log scale.
colors = colormap(normalizer(gdf_population['Population']), bytes=True)

In [ ]:
polygon_layer.get_fill_color = colors

## Add a Legend
To make our map even clearer, we’ll add a colorbar that shows how population densities translate into colors. 
Here’s how we set it up:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 0.8))

# Define the colorbar with LogNorm and the specified colormap
cbar = plt.colorbar(
    plt.cm.ScalarMappable(norm=LogNorm(vmin=gdf_population['Population'].min(), vmax=gdf_population['Population'].max()), cmap=colormap),
    cax=ax,
    orientation='horizontal'
)

tick_values = np.logspace(np.log10(gdf_population['Population'].min()), np.log10(gdf_population['Population'].max()), num=5, base=10.0)
cbar.set_ticks(tick_values)
cbar.ax.xaxis.set_major_formatter(mticker.FuncFormatter(thousands_formatter))
cbar.ax.tick_params(labelsize=8)

cbar.set_label('Population per cell', fontsize=10)

colorbar_output = widgets.Output()

with colorbar_output:
    plt.tight_layout()
    plt.show()

In [ ]:
widgets.VBox([m, colorbar_output])

## Sharing your Map

To share your interactive map, you can save it as an HTML file. This allows others to view and interact with the map without needing any special software. Here’s a simple way to save your map:

In [ ]:
m.to_html("visualization_population.html")

In [ ]:
!open visualization_population.html

However, be aware that saving to HTML means the entire data is embedded in the file. This can result in a large file size, which may not be the most efficient for sharing large datasets.

## Sharing Your Map

Saving as an HTML file embeds all data, which can make the file large and less efficient for big datasets. Another option is to use [`Shiny`](https://developmentseed.org/lonboard/latest/ecosystem/shiny/) which integrates with Jupyter Widgets and supports Lonboard. 

## Interactive Filtering

To make our map even more interactive, we can filter the data based on specific criteria. By using widgets like sliders, we can dynamically adjust which data is displayed. For example, we can set up filters to show only areas with populations within a certain range. 

In [ ]:
filter_extension = DataFilterExtension()

In [ ]:
polygon_layer = SolidPolygonLayer.from_geopandas(
    gdf_population,
    get_fill_color = colors,
    extensions=[filter_extension],
    get_filter_value=gdf_population['Population'], 
    filter_range=[3, 3549] 
)

In [ ]:
with widgets.Output():
    # Create the slider
    slider = widgets.IntRangeSlider(
        value=(3, 25),
        min=3,
        max=3549,
        step=1,
        description="Slider: ",
        layout=widgets.Layout(width='600px'), 
    )

    # Link the slider to the polygon layer 
    widgets.jsdlink(
        (slider, "value"),
        (polygon_layer, "filter_range")
    )

In [ ]:
slider

In [ ]:
m = Map(polygon_layer)
m

# Creating 3D Maps

To add a 3D perspective to our map, we can visualize the polygon layer with elevation. By setting the `extruded` parameter to `True`, we give the polygons height based on population data.

Here’s how it works:

In [ ]:
polygon_layer = SolidPolygonLayer.from_geopandas(
    gdf_population,
    get_fill_color= colors,
    extruded = True
)

In [ ]:
polygon_layer.get_elevation = 1500 * normalizer(gdf_population['Population'])

In [ ]:
m = Map(polygon_layer)
m